In [30]:
import json
from openai import OpenAI

In [31]:
with open('../data/documents.json', 'rt') as f_in:
    documents = json.load(f_in)

In [32]:
documents[0]['question']

'Who found the answer to a search query collar george herbert essay?'

In [33]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [34]:
from elasticsearch import Elasticsearch

In [35]:

es_client = Elasticsearch('http://localhost:9200') 

In [36]:

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "context": {"type": "text"},
            "answer": {"type": "text"},
            "question": {"type": "text"}
        }
    }
}


In [37]:
index_name = "questions-related"
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'questions-related'})

In [38]:
from tqdm.auto import tqdm

In [39]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████| 960/960 [00:23<00:00, 40.77it/s]


In [40]:
def elastic_search(query):
    # search_query = {
    #     "size": 5,
    #     "query": {
    #         "bool": {
    #             "must": {
    #                 "multi_match": {
    #                     "query": query,
    #                     "fields": ["question^3", "context^3", "answer"],
    #                     "type": "best_fields"
    #                 }
    #             }
    #         }
    #     }
    # }
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "context", "answer"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs


In [41]:
query = 'How many percentage of the U.S. retail lingerie market did L-Brands account for in 2024?'

In [42]:
result_docs = elastic_search(query)

In [43]:
for i in range(5):
    
    print(result_docs[i]['answer'])

In 2016, L-Brands, the parent company of Victoria’s Secret, accounted for 83 percent of the U.S. retail lingerie market.
The person paid Rs 19,000 for the bicycle they purchased from Jackson Market.
The person ran 8 miles on Monday.
The parliament met for 135 times during the second session of the Legislature-Parliament.
The Twitter account @0109mahahu started being active since August 2013.


In [44]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "context^3", "answer"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }
    
    # search_query = {
    #     "size": 5,
    #     "query": {
    #         "bool": {
    #             "must": {
    #                 "multi_match": {
    #                     "query": query,
    #                     "fields": ["question^3", "context", "answer"],
    #                     "type": "best_fields"
    #                 }
    #             }
    #         }
    #     }
    # }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs


In [45]:
result_docs = elastic_search(query)

In [46]:
for i in range(5):
    print(result_docs[i]['question'])
    print(result_docs[i]['answer'])
    print("-----------------------")

What percentage of the U.S. retail lingerie market did L-Brands, the parent company of Victoria’s Secret, account for in 2016?
In 2016, L-Brands, the parent company of Victoria’s Secret, accounted for 83 percent of the U.S. retail lingerie market.
-----------------------
What is the usual adult dosage for L-Methyl-MC NAC tablets?
The usual adult dose is one tablet daily or as prescribed by a licensed medical practitioner.
-----------------------
What are the terms of the new five-year pact between Verizon and the NFL?
Under the new five-year pact, Verizon will pay the NFL more than $2 billion to gain rights to live-stream all in-market and national games, including preseason, regular season, and playoff games — including the Super Bowl — to any mobile device nationwide. The deal also covers the NFL’s Thursday, Sunday and Monday Night Football games, and the Sunday day games that air on TV in viewers’ local market. Verizon’s rights extend to mobile phones across all carriers and across 

In [47]:
def build_prompt(query, search_results):
    prompt_template = """
You're a chatbot assistant. Answer the QUESTION based on the CONTEXT from the database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"context: {doc['context']}\nquestion: {doc['question']}\nanswer: {doc['answer']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [48]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [49]:
query = 'How many percentage of the U.S. retail lingerie market did L-Brands account for in 2024?'
rag(query)

InternalServerError: Error code: 500 - {'error': {'message': 'model requires more system memory (5.6 GiB) than is available (2.8 GiB)', 'type': 'api_error', 'param': None, 'code': None}}